# Scrape username from LinkedIn

In [1]:
#from tqdm.notebook import tqdm
from tqdm import tqdm #check for the scarping progress
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from scrape_linkedin import ProfileScraper
from selenium.webdriver.common.by import By

In [2]:
def get_name():
    # get username
#     name_list = []
    #driver.page_source is a entire page's HTML
    soup = BeautifulSoup(driver.page_source,'html.parser') # this step get all the static page info, BeautifulSoup is a object
    
    for a in soup.find(class_="search-results__list list-style-none ").find_all('a', href=True):
        name = a['href'].split('/')[-2]  #a['href'] : /in/anastasisbele/
        if a['href'].split('/')[1] == 'in' and name not in name_list:
            name_list.append(name)
    #batch(name_list)

In [32]:
#not need for now
# save while scraping
import pickle
def batch(name):
    with open('name.pkl', 'ab') as fp: 
        #with open(file_name, mode), use with makes sure the file is closed after the session, 'a' for append, 'b' for binary mode
        pickle.dump(name, fp) #pickle.dump(obj, file), write object into file

In [3]:
#dynamic scraping
driver = webdriver.Chrome()
url = "https://www.linkedin.com/search/results/people/?keywords=data%20analyst&origin=SWITCH_SEARCH_VERTICAL"
driver.get(url)

#login linkedin
driver.find_element_by_css_selector('body > div > main > p > a').click()
time.sleep(0.5)
driver.find_element_by_css_selector('input#username').send_keys('zz629@cornell.edu') # your email
driver.find_element_by_css_selector('input#password').send_keys('Zx16m73yl') # your password for linkedin
driver.find_element_by_css_selector('#app__container > main > div:nth-child(2) > form > div.login__form_action_container > button').click()

for i in tqdm(range(1)): # how many pages do we want to scrape
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll down and wait website to load
    time.sleep(10)
    
    name_list = []
    get_name()  # get name by using Beautiful soup
    time.sleep(5)

    # click on next bottom to go to next page
    xpath = '//button[contains(@aria-label,"Next")]'
    driver.find_element(By.XPATH, xpath).click()

100%|██████████| 1/1 [00:20<00:00, 20.23s/it]


In [34]:
#not need for now
# open name.pkl file
name = []
with open('name.pkl', 'rb') as fr:
    try:
        while True:
            name.append(pickle.load(fr)) 
            #pickle.load(fr): pickle.load(file)
            #read the object from the file and return the reconstituted object hierarchy specified therein
    except EOFError:
        pass

from itertools import chain
name_list = list(chain.from_iterable(name)) #chain thins in different arrays together
name_list

['yuchen-miranda-zhao',
 'brendanrichard',
 'brit-weimer-4096b1a',
 'ziyi-li-333a21168',
 'akira-yamauchi-8b94a3b8',
 'aubreeju',
 'wenting-tong-a9218143',
 'alice-z-3192b32b',
 'michaelvoltmer',
 'yu-wu-9a1716156',
 'james-ci-032448134',
 'yulun-ding-ambassador',
 'yunhan-henry-feng',
 'krystalwang0708',
 'dustin-zamboni-6835a654',
 'shan-lin-0723',
 'jingxuanzhou123',
 'brendanrichard',
 'yvonne-guan-690a4b196',
 'ziwei-dong-uw-smu',
 'akira-yamauchi-8b94a3b8',
 'yunhan-henry-feng',
 'helipatel13',
 'brit-weimer-4096b1a',
 'preethiprakash',
 'echo-gao-4b7a83108',
 'yagao97',
 'xuetinghu',
 'aoli',
 'an-ting-cheng-0227',
 'brendanrichard',
 'yunhan-henry-feng',
 'tim-fox-a81776165',
 'droychowdhury',
 'zixianyang',
 'aubreeju',
 'saumya-arora-63624912b',
 'brit-weimer-4096b1a',
 'jingyu-zhang-38709265',
 'dustin-zamboni-6835a654',
 'saharnasiri',
 'linli-molly-wang-3a5350159',
 'zihao-wang-15820735',
 'jordankohn',
 'wenchen-sandysun',
 'yvonne-guan-690a4b196',
 'sohamjadhav',
 'darel

In [4]:
len(name_list)
name_list

['anastasisbele',
 'paulananya',
 'jamie-yu-3348434b',
 'yun-zhang3384',
 'popov-v',
 'xiaohan-shen-2b909487',
 'fangyuchou1993',
 'anna-mcgovern-75537627',
 'bingdi-chunyu-1535a0b4',
 'zhenwei-hanson-zhang-176a1a114',
 'alissa-wang-85842891']

In [5]:
import csv
LI_AT = 'AQEDASqt89oFOYnbAAABcB2vVKkAAAFxeby1ZU0APL5S3jkundKs8NMku9JqGfod9KTZSvenbZTjqIv5LLffYlAmjiNpGiHad6WIlwDbX2-TfVS56hWIcBUB4wW0CZ2y7uIM_BBcikJXup38mpvfiUwA'

users = ['yuchen-miranda-zhao', 'jamie-yu-3348434b', 'rongxinz', 'ziyi-li-333a21168'] 

with open('some.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for name in name_list:
        with ProfileScraper(cookie = LI_AT) as scraper:
            profile = scraper.scrape(user = name)
        for job in profile.experiences['jobs']:
            writer.writerow([name, job['title'], job['company'], job['date_range'], job['location'], job['description']])
            #print(name, job['title'], job['company'], job['date range'], job['location'], job['description'])

In [6]:
import pandas as pd
df = pd.read_csv('some.csv')
df.columns = ['name', 'title', 'company', 'date_range', 'location', 'description']
df

,name,title,company,date_range,location,description
0,anastasisbele,Operations Research Analyst,Cornell Tech,May 2019 – Sep 2019,Greater New York City Area,•\tDesign optimization software for optimal as...
1,anastasisbele,Project Analyst,Philip Morris International,Jun 2016 – Oct 2017,Neuchâtel,•\tPlanned and executed operational projects f...
2,anastasisbele,Analyst,Humatica,Mar 2016 – May 2016,"Zürich Area, Switzerland",Internship in a boutique consulting firm on st...
3,anastasisbele,Procurement Analyst,Philip Morris International,Jan 2016 – Mar 2016,"Lausanne Area, Switzerland",Analysis of suppliers' performance data from t...
4,anastasisbele,Supply Chain Intern,Philip Morris International,Feb 2015 – Aug 2015,"Lausanne Area, Switzerland","Master Thesis Project: ""Optimizing the Supplie..."
5,anastasisbele,Research Assistant,EPFL (École polytechnique fédérale de Lausanne),Mar 2014 – Jun 2014,"Lausanne Area, Switzerland",Encoding and analysis of data on youth unemplo...
6,anastasisbele,Wired Communication Technician,Hellenic Army,Nov 2012 – Aug 2013,Greece,Military service in the Technical Corps. Train...
7,anastasisbele,Communications Engineer Trainee,Infitheon Technologies,Oct 2011 – Jan 2012,"Athens, Greece",Development and Management of innovative ICT p...
8,anastasisbele,Summer Intern,HELLENIC PETROLEUM Group of Companies,Jul 2010 – Aug 2010,"Aspropyrgos, Greece",Task scheduling under the Maintenance & Progra...
9,paulananya,"Software Engineering Intern, Rider Vertical",Uber,May 2019 – Aug 2019,"Seattle, Washington",As a Software Engineering Intern with the Rese...


In [18]:
#below are random
import pandas as pd
df = pd.DataFrame(result, columns = ['summary', 'skills'])
df.to_csv('out.csv', index=False)

In [13]:
#dynamic scraping
driver = webdriver.Chrome()
url = "https://www.linkedin.com/jobs/search/?keywords=data%20analyst"
driver.get(url)

# login linkedin
driver.find_element_by_css_selector('body > div.cta-modal.show > a.cta-modal__primary-btn').click()
time.sleep(0.5)
driver.find_element_by_css_selector('input#username').send_keys('zz629@cornell.edu') # your email
driver.find_element_by_css_selector('input#password').send_keys('Zx16m73yl') # your password for linkedin
driver.find_element_by_css_selector('#app__container > main > div:nth-child(2) > form > div.login__form_action_container > button').click()

element = driver.find_elements_by_class_name('job-card-search__link-wrapper js-focusable disabled ember-view')
#time.sleep(0.5)
element

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=81.0.4044.92)
